In [1]:
# import the necessary packages
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path

import tkinter as tk

# import the necessary packages
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

from imutils.video import VideoStream
from imutils.video import FPS
import time




In [2]:
#hit space bar to capture
def captureuser(name):
    cam = cv2.VideoCapture(0)

    cv2.namedWindow("capture")

    img_counter = 0
    
    dirname = f'dataset/{name}'
    os.mkdir(dirname)

    while True:
        ret, frame = cam.read()
        cv2.imshow("capture", frame)
        
        if img_counter == 5:
            cv2.destroyWindow("capture")
            break
        if not ret:
            break
        k = cv2.waitKey(1)

        if k%256 == 27:
            # ESC pressed
            print("Escape hit, closing...")
            break
        elif k%256 == 32:
            # SPACE pressed
            path = f'dataset/{name}'
            img_name = "{}.jpg".format(img_counter)
            cv2.imwrite(os.path.join(path , img_name), frame)
            cv2.imwrite(img_name, frame)
            print("{} written!".format(img_name))
            img_counter += 1

    cam.release()

    cv2.destroyAllWindows()

In [3]:
captureuser('tundeAdewole')

0.jpg written!
1.jpg written!
2.jpg written!
3.jpg written!
4.jpg written!


In [17]:
def write_to_csv(unique_id,name,bank, password, account_balance):
    import csv
    
    with open(r'bank_details.csv','a', newline = '\n') as f:
        writer = csv.writer(f)
        writer.writerow([unique_id,name,bank, password, account_balance])

In [18]:
write_to_csv('10234', 'Adewole Tunde 1', 'Gtbank', '12345', '20000.00')

In [19]:
def get_embeddings():
    #summary:
    
    
    
    
    
    
    
    
    
    # construct the argument parser and parse the arguments
    ap = argparse.ArgumentParser()
    ap.add_argument("-i", "--dataset", required=True,
        help="path to input directory of faces + images")
    ap.add_argument("-e", "--embeddings", required=True,
        help="path to output serialized db of facial embeddings")
    ap.add_argument("-d", "--detector", required=True,
        help="path to OpenCV's deep learning face detector")
    ap.add_argument("-m", "--embedding-model", required=True,
        help="path to OpenCV's deep learning face embedding model")
    ap.add_argument("-c", "--confidence", type=float, default=0.5,
        help="minimum probability to filter weak detections")
    #args = vars(ap.parse_args())
    
    # load our serialized face detector from disk
    print("[INFO] loading face detector...")

    detector = cv2.dnn.readNetFromCaffe('face_detection_model/deploy.prototxt', 'face_detection_model/res10_300x300_ssd_iter_140000.caffemodel')
    # load our serialized face embedding model from disk
    embedder = cv2.dnn.readNetFromTorch('openface_nn4.small2.v1.t7')

    # grab the paths to the input images in our dataset
    print("[INFO] quantifying faces...")
    imagePaths = list(paths.list_images('dataset'))
    # initialize our lists of extracted facial embeddings and
    # corresponding people names
    knownEmbeddings = []
    knownNames = []
    # initialize the total number of faces processed
    total = 0
    # loop over the image paths
    for (i, imagePath) in enumerate(imagePaths):
        # extract the person name from the image path
        print("[INFO] processing image {}/{}".format(i + 1,
            len(imagePaths)))
        name = imagePath.split(os.path.sep)[-2]

        # load the image, resize it to have a width of 600 pixels (while
        # maintaining the aspect ratio), and then grab the image
        # dimensions
        image = cv2.imread(imagePath)
        image = imutils.resize(image, width=600)
        (h, w) = image.shape[:2]
        # construct a blob from the image
        imageBlob = cv2.dnn.blobFromImage(
            cv2.resize(image, (300, 300)), 1.0, (300, 300),
            (104.0, 177.0, 123.0), swapRB=False, crop=False)

        # apply OpenCV's deep learning-based face detector to localize
        # faces in the input image
        detector.setInput(imageBlob)
        detections = detector.forward()

        # ensure at least one face was found
        if len(detections) > 0:
            # we're making the assumption that each image has only ONE
            # face, so find the bounding box with the largest probability
            i = np.argmax(detections[0, 0, :, 2])
            confidence = detections[0, 0, i, 2]

            # ensure that the detection with the largest probability also
            # means our minimum probability test (thus helping filter out
            # weak detections)
            if confidence > 0.5:
                # compute the (x, y)-coordinates of the bounding box for
                # the face
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                # extract the face ROI and grab the ROI dimensions
                face = image[startY:endY, startX:endX]
                (fH, fW) = face.shape[:2]

                # ensure the face width and height are sufficiently large
                if fW < 20 or fH < 20:
                    continue

                # construct a blob for the face ROI, then pass the blob
                # through our face embedding model to obtain the 128-d
                # quantification of the face
                faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
                    (96, 96), (0, 0, 0), swapRB=True, crop=False)
                embedder.setInput(faceBlob)
                vec = embedder.forward()
    
                # add the name of the person + corresponding face
                # embedding to their respective lists
                knownNames.append(name)
                knownEmbeddings.append(vec.flatten())
                total += 1
    # dump the facial embeddings + names to disk
    print("[INFO] serializing {} encodings...".format(total))
    data = {"embeddings": knownEmbeddings, "names": knownNames}
    f = open('output/embeddings.pickle', "wb")
    f.write(pickle.dumps(data))
    f.close()


In [20]:
get_embeddings()

[INFO] loading face detector...


error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\dnn\src\torch\THDiskFile.cpp:496: error: (-2:Unspecified error) cannot open <openface_nn4.small2.v1.t7> in mode r  in function 'TH::THDiskFile_new'


In [21]:
def train_model():
    #summary
    
    
    
    
    
    
    print("[INFO] loading face embeddings...")
    data = pickle.loads(open('output/embeddings.pickle', "rb").read())
    le = LabelEncoder()
    labels = le.fit_transform(data["names"])
    # train the model used to accept the 128-d embeddings of the face and
    # then produce the actual face recognition
    print("[INFO] training model...")
    recognizer = SVC(C=1.0, kernel="linear", probability=True)
    recognizer.fit(data["embeddings"], labels)
    # write the actual face recognition model to disk
    f = open('output/recognizer.pickle', "wb")
    f.write(pickle.dumps(recognizer))
    f.close()

    # write the label encoder to disk
    f = open('output/le.pickle', "wb")
    f.write(pickle.dumps(le))
    f.close()
    
    

In [22]:
train_model()

[INFO] loading face embeddings...
[INFO] training model...


In [10]:
def video_check():
    detector = cv2.dnn.readNetFromCaffe('face_detection_model/deploy.prototxt', 'face_detection_model/res10_300x300_ssd_iter_140000.caffemodel')
    #summary
    
    
    
    
    # load our serialized face embedding model from disk
    print("[INFO] loading face recognizer...")
    embedder = cv2.dnn.readNetFromTorch('openface_nn4.small2.v1.t7')

    # load the actual face recognition model along with the label encoder
    recognizer = pickle.loads(open('output/recognizer.pickle', "rb").read())
    le = pickle.loads(open('output/le.pickle', "rb").read())

    # initialize the video stream, then allow the camera sensor to warm up
    print("[INFO] starting video stream...")
    vs = VideoStream(src=0).start()
    time.sleep(2.0)

    #run check for only 15seconds and then stop
    timeout = time.time() + 15
    
    # start the FPS throughput estimator
    fps = FPS().start()

        # loop over frames from the video file stream
    while True:
        #run check for only 15seconds and then stop
        if time.time() > timeout:
            cv2.destroyWindow("Frame")
            break
            
        # grab the frame from the threaded video stream
        frame = vs.read()

        # resize the frame to have a width of 600 pixels (while
        # maintaining the aspect ratio), and then grab the image
        # dimensions
        frame = imutils.resize(frame, width=600)
        (h, w) = frame.shape[:2]

        # construct a blob from the image
        imageBlob = cv2.dnn.blobFromImage(
            cv2.resize(frame, (300, 300)), 1.0, (300, 300),
            (104.0, 177.0, 123.0), swapRB=False, crop=False)

        # apply OpenCV's deep learning-based face detector to localize
        # faces in the input image
        detector.setInput(imageBlob)
        detections = detector.forward()

        #TODO: if 2 faces are detected alert the user of a warning
        # loop over the detections
        for i in range(0, detections.shape[2]):
            # extract the confidence (i.e., probability) associated with
            # the prediction
            confidence = detections[0, 0, i, 2]

            # filter out weak detections
            if confidence > 0.5:
                # compute the (x, y)-coordinates of the bounding box for
                # the face
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                # extract the face ROI
                face = frame[startY:endY, startX:endX]
                (fH, fW) = face.shape[:2]

                # ensure the face width and height are sufficiently large
                if fW < 20 or fH < 20:
                    continue

                # construct a blob for the face ROI, then pass the blob
                # through our face embedding model to obtain the 128-d
                # quantification of the face
                faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
                    (96, 96), (0, 0, 0), swapRB=True, crop=False)
                embedder.setInput(faceBlob)
                vec = embedder.forward()

                # perform classification to recognize the face
                preds = recognizer.predict_proba(vec)[0]
                j = np.argmax(preds)
                proba = preds[j]
                name = le.classes_[j]

                # draw the bounding box of the face along with the
                # associated probability
                text = "{}: {:.2f}%".format(name, proba * 100)
                y = startY - 10 if startY - 10 > 10 else startY + 10
                cv2.rectangle(frame, (startX, startY), (endX, endY),
                    (0, 0, 255), 2)
                cv2.putText(frame, text, (startX, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

        # update the FPS counter
        fps.update()

        # show the output frame
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF

        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break

    # stop the timer and display FPS information
    fps.stop()
    print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

    # do a bit of cleanup
    cv2.destroyAllWindows()
    vs.stop()

In [11]:
video_check()

[INFO] loading face recognizer...


error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\dnn\src\torch\THDiskFile.cpp:496: error: (-2:Unspecified error) cannot open <openface_nn4.small2.v1.t7> in mode r  in function 'TH::THDiskFile_new'


In [23]:
# load our serialized face detector from disk
def recognize(imagePath):
    #summary
    
    
    
    
    
    
    print("[INFO] loading face detector...")
    detector = cv2.dnn.readNetFromCaffe('face_detection_model/deploy.prototxt', 'face_detection_model/res10_300x300_ssd_iter_140000.caffemodel')
    
 
    # load our serialized face embedding model from disk
    print("[INFO] loading face recognizer...")
    embedder = cv2.dnn.readNetFromTorch('nn4.small2.v1.t7')
 
    # load the actual face recognition model along with the label encoder
    recognizer = pickle.loads(open('output/recognizer.pickle', "rb").read())
    le = pickle.loads(open('output/le.pickle', "rb").read())
    
    # load the image, resize it to have a width of 600 pixels (while
    # maintaining the aspect ratio), and then grab the image dimensions
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width=600)
    (h, w) = image.shape[:2]
 
    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)
 
    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()
    
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = detections[0, 0, i, 2]
 
        # filter out weak detections
        if confidence > 0.5:
            # compute the (x, y)-coordinates of the bounding box for the
            # face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
 
            # extract the face ROI
            face = image[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]
 
            # ensure the face width and height are sufficiently large
            if fW < 20 or fH < 20:
                continue
            # construct a blob for the face ROI, then pass the blob
            # through our face embedding model to obtain the 128-d
            # quantification of the face
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96),
                (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()
 
            # perform classification to recognize the face
            preds = recognizer.predict_proba(vec)[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j]
            
            # draw the bounding box of the face along with the associated
            # probability
            text = "{}: {:.2f}%".format(name, proba * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(image, (startX, startY), (endX, endY),
                (0, 0, 255), 2)
            
            #TODO: Handle if 2 faces are given.
            #Decision boundary
            if (name != 'unknown') and (proba *100) < 50:
                print("Fraud detected")
            else:
                print(name)
            cv2.putText(image, text, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
 
    # show the output image
    cv2.imshow("Image", image)
    cv2.waitKey(0)

In [13]:
recognize('images/trisha_adrian.jpg')

[INFO] loading face detector...
[INFO] loading face recognizer...


error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\dnn\src\torch\THDiskFile.cpp:496: error: (-2:Unspecified error) cannot open <openface_nn4.small2.v1.t7> in mode r  in function 'TH::THDiskFile_new'


In [24]:
def enrollment(Fullname, unique_id, password, images, account_balance):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-24-a5d9c1160d32>, line 2)

In [25]:
def homescreen():
    from PIL import ImageTk, Image
    window = tk.Tk()
    window.title("WELCOME")
    label = tk.Label(window, text = "Welcome to GTbank").pack()
    background_image = 'GUI/bg.jpg'
    
   
    
   

    window.mainloop()